In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv, global_max_pool, knn_graph
import torch.optim as optim
from torch_geometric.loader import DataLoader
from tqdm import tqdm


from torch_geometric.data import Data

import os
import shutil
import numpy as np

import random

# Set seed
# SEED=42
# random.seed(SEED)
# torch.manual_seed(SEED)
# np.random.seed(SEED)


In [2]:
DEVICE = torch.device('cuda:0')
K = 4

In [3]:
class PointNetInstanceSeg(nn.Module):
    def __init__(self):
        super(PointNetInstanceSeg, self).__init__()
        self.edge_conv1 = DynamicEdgeConv(nn.Sequential(
            nn.Linear(14, 64),
            nn.SiLU(),
            nn.Linear(64, 128),
            nn.SiLU()
        ), k=K)
        # self.edge_conv2 = DynamicEdgeConv(nn.Sequential(
        #      nn.Linear(256, 512),
        #      nn.SiLU(),
        #      nn.Linear(512, 512),
        #      nn.SiLU()
        #  ), k=8)
        # self.edge_conv3 = DynamicEdgeConv(nn.Sequential(
        #      nn.Linear(1024, 512),
        #      nn.SiLU(),
        #      nn.Linear(512, 256),
        #      nn.SiLU()
        #  ), k=8)
        # self.edge_conv4 = DynamicEdgeConv(nn.Sequential(
        #      nn.Linear(256, 256),
        #      nn.SiLU(),
        #      nn.Linear(256, 128),
        #      nn.SiLU()
        #  ), k=K)
        self.fc = nn.Linear(128, 213)  # Predicting instance mask for each point

    def forward(self, data):
        x, edge_index = data.pos, data.edge_index
        # print(data.edge_index)
        # return
        x = self.edge_conv1(x, edge_index)
        # x = self.edge_conv2(x, edge_index)
        # x = self.edge_conv3(x, edge_index)
        # x = self.edge_conv4(x, edge_index)
        x = self.fc(x)
        return x

    def adjust_fc(self, num_classes):
        self.fc = nn.Linear(128, num_classes = 1)

# Example usage
# model = PointNetInstanceSeg().to(DEVICE)

# Assuming you have point cloud data in a DataLoader
# Example data loading code:
# loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# for data in loader:
#     instance_masks = model(data)
#     # Use instance_masks for further processing or interpretation


# Calc number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(PointNetInstanceSeg())

36757

In [4]:
# Assuming you have point cloud data stored as a list of positions

data = "./Labeled subhalo matrices of haloes/train"
files = os.listdir(data)
point_cloud_data = [(np.load(data+"/"+f)) for f in files if f.endswith(".npy")] # List of point cloud data, each element is a list of point coordinates

# Convert each point cloud data into a Data object
data_list = []
for point_cloud in point_cloud_data:
    # Create a Data object with the positions of points
    pos = torch.tensor(point_cloud[:,:-1], dtype=torch.float)
    # Recentering positions per halo
    pos[:3] = pos[:3] - pos[:3].mean(dim=1, keepdim=True)
    data = Data(
        pos=pos,
        y = torch.tensor(point_cloud[:,-1]+1, dtype=torch.long),
        # edge_index=knn_graph(pos, k=K)
    )
    # data.y = torch.tensor(point_cloud[:,-1], dtype=torch.long)
    # Dynamically generate edge_index based on the positions of points
    # You can use a method like k-NN to construct the edges
    # For example, using knn_graph from torch_geometric.transforms:
    # from torch_geometric.transforms import knn_graph
    # data.edge_index = knn_graph(data.pos, k=K)  # Construct edges based on 6 nearest neighbors
    # Add other necessary attributes to the Data object if needed
    # For example, data.y for ground truth labels
    data_list.append(data)

# Now you can use DataLoader with this list of Data objects
loader = DataLoader(data_list, batch_size=1, shuffle=True)

In [5]:
p = 203

weight_vec = np.repeat(1/p, p)
weight_vec = np.concatenate([np.array([(1/p)*(1/90)]), np.repeat([(1/p)*(1/10)], 9), weight_vec])
weight_vec.shape, weight_vec.sum()

((213,), 1.004488232074439)

In [6]:
# Assuming you have point cloud data stored as a list of positions

data = "./Labeled subhalo matrices of haloes/train"
files = os.listdir(data)
point_cloud_data = [(np.load(data+"/"+f)) for f in files if f.endswith(".npy")] # List of point cloud data, each element is a list of point coordinates

# Convert each point cloud data into a Data object
data_list = []
for point_cloud in point_cloud_data:
    # Create a Data object with the positions of points
    pos = torch.tensor(point_cloud[:,:-1], dtype=torch.float)
    # Recentering positions per halo
    pos[:3] = pos[:3] - pos[:3].mean(dim=1, keepdim=True)
    data = Data(
        pos=pos,
        y = torch.tensor(point_cloud[:,-1]+1, dtype=torch.long),
        # edge_index=knn_graph(pos, k=K)
    )
    # data.y = torch.tensor(point_cloud[:,-1], dtype=torch.long)
    # Dynamically generate edge_index based on the positions of points
    # You can use a method like k-NN to construct the edges
    # For example, using knn_graph from torch_geometric.transforms:
    # from torch_geometric.transforms import knn_graph
    # data.edge_index = knn_graph(data.pos, k=K)  # Construct edges based on 6 nearest neighbors
    # Add other necessary attributes to the Data object if needed
    # For example, data.y for ground truth labels
    data_list.append(data)

# Now you can use DataLoader with this list of Data objects
loader = DataLoader(data_list, batch_size=1, shuffle=True)

In [7]:
# Define your dataset and DataLoader
# dataloader = DataLoader(SHDataSet("train"), batch_size=1, shuffle=True)
# Initialize the model
model = PointNetInstanceSeg().to(DEVICE)
weights = torch.FloatTensor(weight_vec).to(DEVICE)
# Define your loss function and optimizer
# Assuming instance masks are represented as class labels

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=5e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in tqdm(loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        try:
            # print(data.pos.shape, data.pos.size(0), data.pos.numel())
            # assert 2 == 3
            # x.size(0) == batch_x.numel()
            data.to(DEVICE)
            # print(data.y.shape)
            optimizer.zero_grad()
            outputs = model(data)
            # Assuming instance masks are represented as class labels and provided in data.y
            # target = torch.argmax(data.y, dim=1)  # Convert one-hot encoded target to class labels
            # loss = criterion(outputs, target)
            loss = criterion(outputs, data.y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * data.num_graphs
        except Exception as e:
            print(data.y.shape)
            raise e

    epoch_loss = running_loss / len(loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Create the "ckpts" directory if it doesn't exist
import os, time

os.makedirs("ckpts", exist_ok=True)
curr_time = time.strftime("%Y%m%d-%H%M%S")
model_name = f"{curr_time}_pointnet_instance_seg"

# Save the model
torch.save(model.state_dict(), f"./ckpts/{model_name}_model.pth")

Epoch 1/10: 100%|██████████| 140/140 [00:29<00:00,  4.78it/s]


Epoch 1/10, Loss: 2137.5242


Epoch 2/10: 100%|██████████| 140/140 [00:29<00:00,  4.81it/s]


Epoch 2/10, Loss: 822.4704


Epoch 3/10: 100%|██████████| 140/140 [00:29<00:00,  4.81it/s]


...

Epoch 8/10, Loss: 7.2972


Epoch 9/10: 100%|██████████| 140/140 [00:28<00:00,  4.86it/s]


Epoch 9/10, Loss: 6.1281


Epoch 10/10: 100%|██████████| 140/140 [00:29<00:00,  4.71it/s]

Epoch 10/10, Loss: 5.6784


In [14]:
# Assuming you have a DataLoader `test_loader` for your test data
test_loader = DataLoader(data_test_list, batch_size=1, shuffle=False)

# Put the model in evaluation mode

# if not model_name:
    

model.eval()

# Initialize a list to store the predictions
ground_truth_labels = []
predictions = []

# Loop over the test data
with torch.no_grad():
    for data in tqdm(test_loader, desc='Testing'):
        # Move data to the device
        data = data.to(DEVICE)
        
        # Pass the data through the model
        outputs = model(data)
        
        # Get the predicted labels
        _, predicted_labels = torch.max(outputs, 1)
        
        # Store the predictions
        ground_truth_labels.append(data.y.cpu().numpy())
        predictions.append(predicted_labels.cpu().numpy())

# At this point, `predictions` is a list of numpy arrays with the predicted labels for each point cloud in the test set
# You can now compare these predictions to the actual labels to compute your test metrics

Testing: 100%|██████████| 30/30 [00:03<00:00,  8.90it/s]


In [15]:
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have the ground truth labels stored in `ground_truth_labels`
# Calculate accuracy
accs = []
f1s = []
for idx, (gt, pred) in enumerate(zip(ground_truth_labels, predictions)):
    # Checking if the model predicts different labels for different points in the same point cloud
    # if np.unique(pred).shape[0] != 1:
    print(idx, "\t", gt.shape, "\t", np.unique(pred), "\t", np.unique(gt))
    print()
    
    accs.append(accuracy_score(gt, pred))
    f1s.append(f1_score(gt, pred, average='weighted'))

print(f"Mean acc: {np.mean(accs):.4f} \pm {np.std(accs):.4f}")
print(f"Mean F!: {np.mean(f1s):.4f} \pm {np.std(f1s):.4f}")

0 	 (10065,) 	 [  0   3   5   6  13  22  76 130] 	 [0 1 2 3]

1 	 (78716,) 	 [0] 	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]

2 	 (5704,) 	 [0] 	 [0 1 2]

3 	 (134538,) 	 [0] 	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77]

4 	 (14113,) 	 [0] 	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]

5 	 (7083,) 	 [0] 	 [0 1 2 3 4 5]

6 	 (10423,) 	 [0] 	 [0 1 2]

7 	 (6830,) 	 [0] 	 [0 1 2 3 4 5 6]

8 	 (6355,) 	 [0] 	 [0]

9 	 (13648,) 	 [0] 	 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]

10 	 (12394,) 	 [0] 	 [0 1 2 3 4 5 6]

11 	 (7559,) 	 [0] 	 [0 1 2 3 4 5]

12 	 (16444,) 	 [0] 	 [ 0  1  2  3  4  5  6  7  8  9 10]

13 	 (6586,) 	 [  0   2   3   5   6  10  13  14  16  20  28  32  33  39  72  76  94 130] 	 [0 1 2 3]

14 	 (9898,) 	 [0] 	 [0 1 2 3]

15 

In [11]:
# np.unique(ground_truth_labels[0], return_counts=True)
np.unique(ground_truth_labels[0]).shape

(4,)

In [12]:
# np.unique(predictions[0], return_counts=True)
np.unique(predictions[0]).shape

(8,)

In [13]:
this is meant to error out if left uncommented

SyntaxError: invalid syntax (2864247681.py, line 1)

In [ ]:
class SHDataSet(torch.utils.data.Dataset):
    def __init__(self, set):
        if set == "train":
            data = "/home/group10/ml/Labeled subhalo matrices of haloes/train"
        elif set == "val":
            data = "/home/group10/ml/Labeled subhalo matrices of haloes/train/val"
        
        self.length = len(data)

        self.set = set
        
        files = os.listdir(data)
        files = [torch.tensor(np.load(data+"/"+f), dtype=torch.float64) for f in files if f.endswith(".npy")]
        files1=[f[:,:-1] for f in files]
        self.files = files1
        
        labels = [f[:,-1] for f in files]
        
        labels = [torch.nn.functional.one_hot(j, 213) for j in labels]
        
        self.labels = labels
        
        
    def __getitem__(self,index):
        return torch.tensor(self.files[index], dtype = torch.float64), torch.tensor(self.labels[index], dtype = torch.long)
        
        
    def __len__(self):
        return self.length

def train_accuracy(
    model,
    data_generator,
    GPU = torch.device("cuda:2"),
):
    model.eval()
    with torch.no_grad():
        accs = []
        for batch_x, batch_y in data_generator:
            batch_x, batch_y = batch_x.to(GPU), batch_y
            y_true = batch_y.argmax(1).numpy()
            y_pred = model(batch_x).argmax(1).cpu().numpy()
            acc = accuracy_score(y_true, y_pred)
            accs.append(acc*100)
    model.train()
    return np.array(accs).mean()

In [ ]:
def __getitem__(self, idx):
    data = np.load(os.path.join(self.directory, self.files[idx]))
    points = torch.tensor(data[:, :3], dtype=torch.float32)  # Assuming the first 3 columns are the point coordinates
    labels = torch.tensor(data[:, 3:], dtype=torch.long)  # Assuming the rest of the columns are the labels
    return points, labels

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_point_cloud(points, labels):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=labels, cmap='jet')
    plt.show()

# Get a batch of data
data = next(iter(loader))

# Run the model and get the outputs
model.eval()
with torch.no_grad():
    outputs = model(data)

# Get the predicted labels
_, predicted_labels = torch.max(outputs, 1)

# Visualize the point cloud with the predicted labels
visualize_point_cloud(data.pos.cpu().numpy(), predicted_labels.cpu().numpy())

In [ ]:
import time

In [ ]:
for data in loader:
    print(data)
    print(data.y.shape)
    print(data.pos.shape)
    break

In [ ]:
# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Loss function expects dynamic number of classes
# optimizer = optim.Adam(model.parameters(), lr=0.001)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
max_class_index = 0  # Initialize maximum class index
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in tqdm(loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        optimizer.zero_grad()
        outputs = model(data)
        
        # Determine the maximum class index encountered
        max_class_index = max(max_class_index, torch.max(data.y).item())
        
        # Adjust the fully connected layer dynamically based on the maximum class index
        model.adjust_fc(max_class_index + 1)  # Add 1 to account for zero-based indexing
        
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.num_graphs
    
    epoch_loss = running_loss / len(loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# After training, you can use the model for inference

In [ ]:
(outputs.shape, data.y.shape)